In [3]:
import pandas as pd
import subprocess
import numpy as npdef send_mail(name,receveur="ari.kouassi@stat.plan.gouv.ci",cc="do.akadie@stat.plan.gouv.ci;j.migone@stat.plan.gouv.ci"):
    ol=win32com.client.Dispatch("outlook.application")
    current_day = datetime.now().strftime("%Y-%m-%d")
    current_hour = datetime.now().strftime("%H")
    olmailitem=0x0 #size of the new email
    newmail=ol.CreateItem(olmailitem)
    newmail.Subject= 'Testing Mail Suivi Carto Automatique'
    newmail.To=receveur
    newmail.CC=cc
    newmail.Body= 'Bonjour M. Aristide, je vous prie de trouver, ci-joint, le point de la vérification cartographique du ' + current_day + ' à ' + current_hour
    attach= wd + name + ".xlsx"
    newmail.Attachments.Add(attach)
    newmail.Send()
import os
from datetime import datetime
import geopandas as gpd
import win32com.client
from shapely.geometry import Point, Polygon

In [15]:
# définition du dossier de travail
# les données sont dans le dossier drive de la reforme
wd = "C:\\Users\\" + os.environ.get('USERNAME') + "\\OneDrive - GOUVCI\\Refonte Enquête Emploi_ Equipe Technique - Fichiers\\Applis\\Suivi_Cartographique\\Suivi_Carto_Automatic\\"

In [16]:
#Lecture des données ménages 
base_verif_carto = pd.read_stata(wd + "Data\\base_menage_carto_verif.dta")

In [17]:
#Lecture des données dénombrement
base_denombrement_batiment = pd.read_stata(wd + "Data\\batiment_verif.dta")

In [33]:
def send_mail(name,receveur="ari.kouassi@stat.plan.gouv.ci",cc="do.akadie@stat.plan.gouv.ci;j.migone@stat.plan.gouv.ci"):
    ol=win32com.client.Dispatch("outlook.application")
    current_day = datetime.now().strftime("%Y-%m-%d")
    current_hour = datetime.now().strftime("%H")
    olmailitem=0x0 #size of the new email
    newmail=ol.CreateItem(olmailitem)
    newmail.Subject= 'Testing Mail Suivi Carto Automatique'
    newmail.To=receveur
    newmail.CC=cc
    newmail.Body= 'Bonjour M. Aristide, je vous prie de trouver, ci-joint, le point de la vérification cartographique du ' + current_day + ' à ' + current_hour
    attach= wd + name + ".xlsx"
    newmail.Attachments.Add(attach)
    newmail.Send()

In [28]:
base_denombrement_batiment.columns

Index(['interview__key', 'interview__id', 'ilot__id', 'batiment__id',
       'list_bat', 'index_bat', 'lib_bat', 'batim', 'gps__Latitude',
       'gps__Longitude',
       ...
       'numero_chef', 'numero_autre_membre', 'menobs', 'obs', 'nb_men_bt',
       '_merge', 'nb_men_ilot', 'nb_men_ik', 'nb_men_zd_denom', 'NumSeg'],
      dtype='object', length=269)

##### Etape 1 : Vérifier que le point ménage est dans le segment : 
###### i. On crée un buffer de 5 m autour du segment ; 
###### ii. On vérifie si le point ménage appartient au buffer

In [29]:
# Construction des points geopandas ménage
base_verif_carto["NumSeg"] = base_verif_carto["HH7"]
base_verif_carto_gpd = gpd.GeoDataFrame(
    base_verif_carto, geometry=gpd.points_from_xy(base_verif_carto['GPS__Longitude'],base_verif_carto['GPS__Latitude']),crs="EPSG:4326")
base_verif_carto_gpd["points_geo"] = base_verif_carto_gpd["geometry"]
by_segment = base_verif_carto_gpd.groupby('HH7')

In [30]:
# Lecture du Shapefile
gdf = gpd.read_file(wd + "SHP\\SEG_Emploi.shp")

In [31]:
#Changement du système de coordonnées en mêtre CIV
gdf = gdf.to_crs(32630)
#Ajout du buffer de 5 metres
gdf['geometry'] = gdf['geometry'].buffer(5)
#Retour aux systèmes du coordonnées en format GPS
gdf = gdf.to_crs(4326)
#Creation d'une variable pour les polygones
gdf["polygon"] = gdf["geometry"]

In [34]:
#Pour chaque segment, on vérifie que les points ménages du segment appartiennent au buffer du segment
for name, grp in by_segment:
    group = grp.merge(gdf, how='inner', on='NumSeg')
    group["within"] = np.nan
    for i in range(len(group)):
        point   = group.loc[i, "points_geo"]
        polygon = group.loc[i, "polygon"]
        within  = point.within(polygon)
        group.loc[i, "within"] = within
    group.to_excel(wd + name + ".xlsx",sheet_name=name)
    send_mail(name)

##### Etape 2 : Vérifier que le point ménage est dans le polygone formé par les points bâtiments : 
###### i.    On fusionne les données pour retrouver les points bâtiments ; 
###### ii.   On crée un polygone basée sur les points bâtiments ;
###### iii.  On vérifie que le point ménage appartient au polygone crée en (ii)

In [24]:
# Construction des points geopandas ménage
base_denombrement_batiment["NumSeg"] = base_denombrement_batiment["Sg"]
base_denombrement_batiment_gpd = gpd.GeoDataFrame(
    base_denombrement_batiment, geometry=gpd.points_from_xy(base_denombrement_batiment['gps__Longitude'],base_denombrement_batiment['gps__Latitude']),crs="EPSG:4326")
base_denombrement_batiment_gpd["bats_geo"] = base_denombrement_batiment_gpd["geometry"]
by_segment = base_denombrement_batiment_gpd.groupby('Sg')

C:\Users\fajmi\AppData\Local\Temp\ipykernel_29260\1635790959.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  base_denombrement_batiment["NumSeg"] = base_denombrement_batiment["Sg"]


In [25]:
bat_list_point = []
for name, grp in by_segment:
    group["polygon_batiment"] = np.nan
    print(name)
    for i in range(len(group)):
        print(i)
        bat = group.loc[i, "bats_geo"]
        print(i)
        bat_list_point.append(bat)


0151_02
0517_03
0751_02
6002_01
6011_01
6011_02
6045_01


In [26]:
bat_list_point

[]